# Structbio2022 - Practical day 02

## Equilibration of the simulation

Document written by [Adrián Diaz](mailto:adrian.diaz@vub.be) & [David Bickel](mailto:david.bickel@vub.be).
   
<br/>

During the **preparation** we built our molecular system. We generated a **coordinate** and a **topology** file. These files contain all the information relevant to the simulation, *e.g.*:
* Atoms to be simulated
* Coordinates of these atoms
* Connectivity (bonds between the atoms)
* Parameters to simulate non-bonded interactions (electrostatic & van-der-Waals interactions )

Before, we start the simulation, lets recapitulate some basics.
The simulation process is carried out in time steps, where each time step corresponds to the passing of a tiny amount of time, $(dt = 2 \cdot 10^{-15} s)$.
During the simulation, each particle has a velocity assigned $(\vec{v})$.
At the beginning of each **time step** ($dt$), the direction an magnitude of $\vec{v}$ gets updated according to the forces acting on the particle (*e.g.*, bonds, neighboring particles, ...). Then, the particle is moved along $\vec{v}$ for $dt$.

So far, none of the atoms in the system have a velocity. THey are perfectly motionless (comparable to bein "frozen" at 0 K). Therefore, we need to "heat" the system up, to get the particles to move at a **physiological temperature**. With increasing the temprature the pressure inside the periodic boundary box will rise. So, we will finally adjust the size of the box, to maintain a **constant pressure** at 1 atm.

***

Let's start by going to the equilibration directory. Then, define the location of the output files from the preparation.

***

## Energy minimization

To remove artifacts that result from low-resolution (crystal) structures and the packaging of the system, we run an energy minimization first.
The energy minimization reduces stress (*e.g.*, by bond streching or clashes), and allows us to start the simulation in a low energetic state.

> #### *What would happen, if the simulation is started from a high-energy state?* ####
> How does the initial energy relate to forces? 
> How do forces relate to particle velocities? 
> What does the the simulation time step mean?

The file `01_min_all.mdp` contains all the parameters for the minimization process. Open the file, and check the parameters.

> #### *How many minimization steps will be performed* ####
> Open `01_min_all.mdp`, and see if you can find out.

Finally, execut the commands below in a terminal.

Lets have a look at the output...

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import panedr
%matplotlib inline

# Read the data
data = panedr.edr_to_df("01_min_all.edr")

fig, ax = plt.subplots(1,1, figsize=(4,2), dpi=150)
ax.grid(True, "both", "both", linewidth=.5, alpha=.5)

# Plot the data
ax.plot(data["Time"], data['Potential'])

ax.set_xlabel("Minimization steps")
plt.show()

***

## Thermalization

Next, we need to **heat** our system up to 298 K.

> #### *Why do we simulate at 298 K?* ####

Since the particles have no velocities yet, at the beginning of this simulation, each particle is assigned a velocity from the Boltzmann distribution at 298 K. Since this assignment is random, we need to simulate a while until the temperature equilibrates around our target temperature.

> #### *How long do we simulate?* ####
> All simulation parameters are written in the `02_nvt_heat.mdp`.

Now, run the simulation on the cluster. This may take a few minutes.

Let's have a look at the evolution of temperature over the simulation time.

Instead of Temperature and Time you can display a variety of properties:
```python
[
    'Time', 'Bond', 'U-B', 'Proper Dih.', 'Improper Dih.', 'CMAP Dih.', 
    'LJ-14', 'Coulomb-14', 'LJ (SR)', 'Coulomb (SR)', 'Coul. recip.', 
    'Potential', 'Pressure', 'Vir-XX', 'Vir-XY', 'Vir-XZ', 'Vir-YX',
    'Vir-YY', 'Vir-YZ', 'Vir-ZX', 'Vir-ZY', 'Vir-ZZ', 'Pres-XX', 'Pres-XY',
    'Pres-XZ', 'Pres-YX', 'Pres-YY', 'Pres-YZ', 'Pres-ZX', 'Pres-ZY',
    'Pres-ZZ', '#Surf*SurfTen', 'T-rest'
]
```

In [ ]:
# Read the data
data = panedr.edr_to_df("02_nvt_heat.edr")

fig, ax = plt.subplots(1,1, figsize=(4,2), dpi=150)
ax.grid(True, "both", "both", linewidth=.5, alpha=.5)

# Plot the data
ax.plot(data["Time"], data['Temperature'])

ax.set_ylabel("Temperature | K")
ax.set_xlabel("Simulation time | ps")
plt.show()

***

## Pressure equilibration

The previous thermalization step was done under constant volume conditions. That means, we kept the size of the periodic boundary box fixed. However, as particles start moving, the pressure inside the box changes. Therefore, we need to adjust the volume to maintain a physiological pressure.

> #### *How did the pressure change during the thermalization?* ####
> Make an educated guess. Then change the code above to show the pressure instead of temperature.

Now, run the simulation on the cluster. This may take a few minutes.

Let's have a look at the results again.

> #### *Did the pressure reach the desired 1 atm?* ####
> Have a look at the `Time` vs `Pressure` plot. Discuss.

> #### *How did the volume change over the course of the simulation? What is the final volume?* ####
> Make and eductated guess. Then plot `Time` vs `Volume` to see, how it actually changed.

In [ ]:
data = panedr.edr_to_df("03_npt_equi.edr")

fig, ax = plt.subplots(1,1, figsize=(4,2), dpi=150)
ax.grid(True, "both", "both", linewidth=.5, alpha=.5)
ax.plot(data["Time"], data['Pressure'])
ax.set_ylabel("Pressure | bar")
ax.set_xlabel("Simulation time | ps")
plt.show()

Finally, let's have a look, how the simulation looks.

In [ ]:
import MDAnalysis as mda
from MDAnalysis.transformations import unwrap, center_in_box
import nglview as nv

MD = mda.Universe("03_npt_equi.tpr", "03_npt_equi.xtc")
MD.trajectory.add_transformations(unwrap(MD.atoms))

In [ ]:
view = nv.show_mdanalysis(MD)
view._remote_call("setSize", target="Widget", args=["800px", "600px"])
view.add_cartoon("protein", color_scheme="sstruc")
view.add_representation("ball+stick", selection="ion")
view.add_surface("all", opacity=0.1)
view.center() # Center and zoom molecule
view

***

## Proceeding to production

With that all preparation of the simulations are complete. We can now start the simulations to answer our questions.
Such simulations typically run for days or weeks on a high-performace computing cluster. Therefore we will submit them to a queuing system and analyze them next week.

To do that run the following commands in a terminal: